# Finite Size Scaling for Time Power laws over Temperature:  

FSS ansatz, 
`scaling variable`, $u(\beta, N) = (\beta-\beta_c) N^a$,      
and `scaling function` $f(u)=A+B \cdot (c \cdot u+d)^{-1}$,   
then power-law exponents $\alpha$   
then `scaling ansatz` $\alpha(\beta, N) = N^{-b} f(u)$.   

$\beta$ inverse temperature. 

Summary of results
Fitted parameters:
```text
Fitted parameters:
a (FSS exponent): 0.294 
b (prefactor): 0.2318 
c (power): 1.2526 
A (constant): 0.2647 
B (amplitude): 8.3646 
d (cons): -2.9509 
```

In [1]:
df <- readRDS("../datasets/ising1DrateErgodicityPowerlawsTime.rds")
df <- df[df["dynamics"] == "Glauber", ]
# alpha, beta, N
df <- df[,c('dynamics', 'alpha_mean', 'ikBT', 'N'),]
N <- as.numeric(df['N']$N)
beta <- as.numeric(df['ikBT']$ikBT)
alpha <- as.numeric(df['alpha_mean']$alpha_mean)
critical_temp_index <- which(alpha == max(alpha))
beta_c <- beta[critical_temp_index]
beta_c

[1] 1.1

In [2]:
N

[1]  512  512  512  512  512  512  512  512  512  512  512  512  512  512  512
[16]  512 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024
[31] 1024 1024 1536 1536 1536 1536 1536 1536 1536 1536 1536 1536 1536 1536 1536
[46] 1536 1536 1536

In [3]:
data <- data.frame(
  "m" = alpha,
  "T" = beta,
  "N" = N
)

T_c <- 1.1 # beta_c

fss_loss <- function(params, data) {
  a <- params[1]
  b <- params[2]
  c <- params[3]
  A <- params[4]
  B <- params[5]
  d <- params[6]

  N <- data$N
  T <- data$T
  m <- data$m
  u <- (T - 1.1) * Re(as.complex(N)^a)
  pred <-  Re(as.complex(N)^-b) * (A + B / (1.0 + exp(-c*Re(as.complex(u))+d)))
  return(sum((m - pred)^2))
}

initial_guess <- c(0.1, 0.1, 0.1, 1.0, 1.0, 1.0, 1.0)


In [4]:
fit <- optim(
  par = initial_guess,
  fn = fss_loss,
  data = data,
  method = "Nelder-Mead"
)

In [5]:


# Extract best parameters
a_fit <- fit$par[1]
b_fit <- fit$par[2]
c_fit <- fit$par[3]
A_fit <- fit$par[4]
B_fit <- fit$par[5]
d_fit <- fit$par[6]

cat("Fitted parameters:\n")
cat("a (FSS exponent):", round(a_fit, 4), "\n")
cat("b (prefactor):", round(b_fit, 4), "\n")
cat("c (power):", round(c_fit, 4), "\n")
cat("A (constant):", round(A_fit, 4), "\n")
cat("B (amplitude):", round(B_fit, 4), "\n")
cat("d (cons):", round(d_fit, 4), "\n")




Fitted parameters:
a (FSS exponent): 0.294 
b (prefactor): 0.2318 
c (power): 1.2526 
A (constant): 0.2647 
B (amplitude): 8.3646 
d (cons): -2.9509 


In [6]:
fx  <- function(N, beta, a, b, c, A, B, d) { # alpha(N,T)
 (beta - 1.1) * Re(as.complex(N)^a) # u
}

fy  <- function(N, beta, a, b, c, A, B, d) { # alpha(N,T)
  u <- (beta - 1.1) * Re(as.complex(N)^a) # u
  (A + B / (1.0 + exp(-c*Re(as.complex(u))+d))) # alpha N^b
}

In [7]:
ll <- length(beta) 
XX <- vector("numeric", ll)
YY <- vector("numeric", ll)
for(i in 1:ll) {
    XX[i] <- fx(N[i], beta[i], a_fit, b_fit, c_fit, A_fit, B_fit, d_fit)
    YY[i] <- fy(N[i], beta[i], a_fit, b_fit, c_fit, A_fit, B_fit, d_fit)
}
# y(x, L) = L−b f [(x− xc)La], # y is alpha



In [8]:

pdf("../plots/finiteSizeScaling/finite_size_overlapping_time_glauber_ansatz_scaled.pdf", pointsize = 18) # pointsize will effect all label and title font sizes.

ix <- which(N == 512)
plot(XX[ix], YY[ix],  col="black", type='p', pch=1,ann=FALSE, xlim=c(-5.0, 5.0), ylim=c(0.0,  10.0))
ix <- which(N == 1024)
lines(XX[ix], YY[ix], col="blue", type="p", pch=2, ann=FALSE, xlim=c(-0.2, 1.1), ylim=c(0.0,  2.2))
ix <- which(N == 1536)
lines(XX[ix], YY[ix], col="red", type='p', pch=3, ann=FALSE, xlim=c(0.0, 1.1), ylim=c(0.0,  2.2))
ylabel <-  bquote(paste("Scaling function f(u):  ",  A + B/(1.0 + exp(-c*u+d))))
xlabel <-  bquote(paste("Scaling variable  ",  u))
title_main <- bquote(paste("Finites Size Scaling for exponent ", Kappa(t), " ~ ", Ct^{alpha}))
subtitle <- bquote(paste(
    "Average Magnetisation Ergodicity : Data collapse"
))
title(title_main, xlab=xlabel, ylab=ylabel)
mtext(subtitle)
    legend("bottomright",
    legend = c("Ansatz at N=512", "Ansatz at N=1024", "Ansatz at N=1536"),
    pch = c(1, 2, 3),
    col = c("black", "blue", "red"),
    lwd = 2,
    bty = "n",
    )
dev.off()

agg_record_19257265 
                  2

In [9]:

pdf("../plots/finiteSizeScaling/finite_size_overlapping_time_glauber_ansatz_unscaled.pdf", pointsize = 18) # pointsize will effect all label and title font sizes.

ix <- which(N == 512)
plot(beta[ix], alpha[ix],  col="black", type='p', pch=1,ann=FALSE, xlim=c(0.4, 1.6), ylim=c(0.0,  2.1))
ix <- which(N == 1024)
lines(beta[ix], alpha[ix], col="blue", type="p", pch=2,ann=FALSE, xlim=c(0.4, 1.6), ylim=c(0.0,  2.1))
ix <- which(N == 1536)
lines(beta[ix], alpha[ix], col="red", type='p',, pch=3,ann=FALSE, xlim=c(0.4, 1.6), ylim=c(0.0,  2.1))
ylabel <-  bquote(paste("Time power-law exponent  ",  alpha(beta, N)))
xlabel <-  bquote(paste("Inverse Temperature  ",  beta))
title_main <- bquote(paste("Unscaled ", Kappa(t), " ~ ", Ct^{alpha}))
subtitle <- bquote(paste(
    "Average Magnetisation Ergodicity : Unscaled data"
))
title(title_main, xlab=xlabel, ylab=ylabel)
mtext(subtitle)
    legend("bottomright",
    legend = c("N=512", "N=1024", "N=1536"),
    pch = c(1, 2, 3),
    col = c("black", "blue", "red"),
    lwd = 2,
    bty = "n",
    )
dev.off()

agg_record_1318754770 
                    2